In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from utils import *

from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import load_dataset, Dataset
import torch
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

In [3]:
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True, max_length=250)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average=None)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1.mean(),
        'precision': precision.mean(),
        'recall': recall.mean()
    }

def acc_at_k(y_true, y_pred, k=2):
    y_true = torch.tensor(y_true) if type(y_true) != torch.Tensor else y_true
    y_pred = torch.tensor(y_pred) if type(y_pred) != torch.Tensor else y_pred
    total = len(y_true)
    y_weights, y_idx = torch.topk(y_true, k=k, dim=-1)
    out_weights, out_idx = torch.topk(y_pred, k=k, dim=-1)
    correct = torch.sum(torch.eq(y_idx, out_idx) * y_weights)
    acc = correct / total
    return acc.item()

def CEwST_loss(logits, target, reduction='mean'):
    """
    Cross Entropy with Soft Target (CEwST) Loss
    :param logits: (batch, *)
    :param target: (batch, *) same shape as logits, each item must be a valid distribution: target[i, :].sum() == 1.
    """
    logprobs = torch.nn.functional.log_softmax(logits.view(logits.shape[0], -1), dim=1)
    batchloss = - torch.sum(target.view(target.shape[0], -1) * logprobs, dim=1)
    if reduction == 'none':
        return batchloss
    elif reduction == 'mean':
        return torch.mean(batchloss)
    elif reduction == 'sum':
        return torch.sum(batchloss)
    else:
        raise NotImplementedError('Unsupported reduction mode.')

def compute_metrics_w_soft_target(pred):
    labels = pred.label_ids
    preds = pred.predictions
    acc = acc_at_k(labels, preds, k=2)
    return {
        'accuracy': acc,
    }

class Trainer_w_soft_target(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs[0]
        loss = CEwST_loss(logits, labels)
        if return_outputs:
            return loss, outputs
        return loss
    
class DefaultCollator:
    def __init__(self):
        pass
    def __call__(self, batch):
        return torch.utils.data.dataloader.default_collate(batch)

In [4]:
MODEL_NAMES = ['bert-base-uncased'] #, 'xlnet-base-cased']

In [5]:
from sklearn.datasets import fetch_20newsgroups

def get_20NG_test_dataset():
    cats = [
        'talk.politics.mideast',                                # Wolrd 0
        'rec.sport.hockey', 'rec.sport.baseball',               # Sports 1
        # 'misc.forsale',                                       # Business 2
        'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', # Sci/Tech 3
    ]

    dataset = fetch_20newsgroups(
        subset='all',
        categories=cats,
        remove=('headers', 'footers', 'quotes')
    )

    df = pd.DataFrame([dataset.data, dataset.target]).T
    df.rename(columns={0:'text', 1: 'label'}, inplace=True)

    mapper = {
        0: 1,
        1: 1,
        2: 3,
        3: 3,
        4: 3,
        5: 3,
        6: 0,
    }

    df.label = df.label.map(mapper)
    df.text = df.text.replace('\n', ' ', regex=True).str.strip()

    test_dataset = Dataset.from_pandas(df)
    
    return test_dataset

In [6]:
use_pretrain = False
soft_target = False

for t in ['ORIG', 'INV', 'SIB-mix', 'INVSIB']:
    for MODEL_NAME in MODEL_NAMES:
        
        eval_only = False
        
        checkpoint = 'pretrained/' + MODEL_NAME
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)    
        
        if t == 'ORIG':
            train_dataset = load_dataset('ag_news')['train']
            checkpoint += '-ag_news-ORIG'
            if use_pretrain and os.path.exists(checkpoint):
                print('loading {}...'.format(checkpoint))
                MODEL_NAME = checkpoint
                eval_only = True
        if t == 'INV':
            text = npy_load("./assets/AG_NEWS/topic/INV/text.npy")
            label = npy_load("./assets/AG_NEWS/topic/INV/label.npy")
            df = pd.DataFrame({'text': text, 'label': label})
            df.text = df.text.astype(str)
            df.label = df.label.astype(int)
            train_dataset = Dataset.from_pandas(df)
            checkpoint += '-ag_news-INV'
            if use_pretrain and os.path.exists(checkpoint):
                print('loading {}...'.format(checkpoint))
                MODEL_NAME = checkpoint
                eval_only = True
        if t == 'SIB-mix':
            text = npy_load("./assets/AG_NEWS/topic/SIB-mix/text.npy")
            label = npy_load("./assets/AG_NEWS/topic/SIB-mix/label.npy")
            df = pd.DataFrame({'text': text, 'label': label.tolist()})
            df.text = df.text.astype(str)
            df.label = df.label.map(lambda y: np.array(y))
            train_dataset = Dataset.from_pandas(df)    
            checkpoint += '-ag_news-SIB-mix'
            if use_pretrain and os.path.exists(checkpoint):
                print('loading {}...'.format(checkpoint))
                MODEL_NAME = checkpoint  
                eval_only = True
        if t == 'INVSIB':
            text = npy_load("./assets/AG_NEWS/topic/INVSIB/text.npy")
            label = npy_load("./assets/AG_NEWS/topic/INVSIB/label.npy")
            df = pd.DataFrame({'text': text, 'label': label.tolist()})
            df.text = df.text.astype(str)
            df.label = df.label.map(lambda y: np.array(y))
            train_dataset = Dataset.from_pandas(df)    
            checkpoint += '-ag_news-INVSIB'
            if use_pretrain and os.path.exists(checkpoint):
                print('loading {}...'.format(checkpoint))
                MODEL_NAME = checkpoint 
                eval_only = True
                
        dataset_dict = train_dataset.train_test_split(
            test_size = 0.1,
            train_size = 0.9,
            shuffle = True
        )
        train_dataset = dataset_dict['train']
        eval_dataset = dataset_dict['test']
        test_dataset = load_dataset('ag_news')['test']
        test_dataset_20NG = get_20NG_test_dataset()
        
        # # reduce training time
        # n = 100
        # train_dataset = Dataset.from_dict(train_dataset[:n])
        # eval_dataset = Dataset.from_dict(eval_dataset[:n])
        # test_dataset = Dataset.from_dict(test_dataset[:n])

        model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=4).to(device)
                
        train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
        eval_dataset = eval_dataset.map(tokenize, batched=True, batch_size=len(eval_dataset))
        test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))
        test_dataset_20NG = test_dataset_20NG.map(tokenize, batched=True, batch_size=len(test_dataset_20NG))
        train_dataset.rename_column_('label', 'labels')
        eval_dataset.rename_column_('label', 'labels')
        test_dataset.rename_column_('label', 'labels')
        test_dataset_20NG.rename_column_('label', 'labels')
        train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
        eval_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
        test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
        test_dataset_20NG.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
        
        if len(np.array(train_dataset['labels']).shape) > 1:
            soft_target = True

        train_batch_size = 3
        eval_batch_size = 32
        num_epoch = 3
        max_steps = int((len(train_dataset) * num_epoch) / train_batch_size)
        
        training_args = TrainingArguments(
            output_dir=checkpoint,
            overwrite_output_dir=True,
            max_steps=max_steps,
            save_steps=int(max_steps / 10),
            save_total_limit=1,
            per_device_train_batch_size=train_batch_size,
            per_device_eval_batch_size=eval_batch_size,
            warmup_steps=int(max_steps / 10),
            weight_decay=0.01,
            logging_dir='./logs',
            logging_steps=int(max_steps / 10),
            load_best_model_at_end=True,
            metric_for_best_model="loss",
            greater_is_better=False,
            evaluation_strategy="steps",
            label_names=['World', 'Sports', 'Business', 'Sci/Tech']
        )
        
        if soft_target:
            trainer = Trainer_w_soft_target(
                model=model,
                args=training_args,
                compute_metrics=compute_metrics_w_soft_target,
                train_dataset=train_dataset,
                eval_dataset=eval_dataset,
                data_collator=DefaultCollator(),
                callbacks=[EarlyStoppingCallback(early_stopping_patience=10)]
            )
        else: 
            trainer = Trainer(
                model=model,
                args=training_args,
                compute_metrics=compute_metrics,
                train_dataset=train_dataset,
                eval_dataset=test_dataset,
                callbacks=[EarlyStoppingCallback(early_stopping_patience=10)]
            )

        if not eval_only:
            trainer.train()
            
        trainer.compute_metrics = compute_metrics
        
        # test with ORIG data
        trainer.eval_dataset = test_dataset
        out_orig = trainer.evaluate()
        print('ORIG \n', out_orig)
        
        # test with 20NG data
        trainer.eval_dataset = test_dataset_20NG
        out_20NG = trainer.evaluate()
        print('20NG \n', out_20NG)

Using custom data configuration default
Reusing dataset ag_news (C:\Users\Fabrice\.cache\huggingface\datasets\ag_news\default\0.0.0\fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a)
Loading cached split indices for dataset at C:\Users\Fabrice\.cache\huggingface\datasets\ag_news\default\0.0.0\fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a\cache-d83824437fa96733.arrow and C:\Users\Fabrice\.cache\huggingface\datasets\ag_news\default\0.0.0\fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a\cache-19c3de4609a5743e.arrow
Using custom data configuration default
Reusing dataset ag_news (C:\Users\Fabrice\.cache\huggingface\datasets\ag_news\default\0.0.0\fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a)
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.p

C:\Users\Fabrice\AppData\Local\Continuum\anaconda3\envs\python38\lib\site-packages\datasets\arrow_dataset.py:850: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  return torch.tensor(x, **format_kwargs)


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Runtime,Samples Per Second
10800,0.517000,0.408373,0.907500,0.907583,0.908603,0.907500,116.664100,65.144000
21600,0.689900,1.391227,0.250000,0.100000,0.062500,0.250000,113.761100,66.807000
32400,1.405000,1.397380,0.250000,0.100000,0.062500,0.250000,114.015800,66.657000
43200,1.403700,1.387325,0.250000,0.100000,0.062500,0.250000,114.179800,66.562000
54000,1.400600,1.390040,0.250000,0.100000,0.062500,0.250000,114.428200,66.417000
64800,1.399300,1.397552,0.250000,0.100000,0.062500,0.250000,115.855100,65.599000
75600,1.397800,1.387144,0.250000,0.100000,0.062500,0.250000,115.205800,65.969000
86400,1.396800,1.389220,0.250000,0.100000,0.062500,0.250000,115.267900,65.933000
97200,1.394700,1.388414,0.250000,0.100000,0.062500,0.250000,115.186600,65.980000
108000,1.393200,1.386345,0.250000,0.100000,0.062500,0.250000,114.090300,66.614000


C:\Users\Fabrice\AppData\Local\Continuum\anaconda3\envs\python38\lib\site-packages\sklearn\metrics\_classification.py:1222: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Fabrice\AppData\Local\Continuum\anaconda3\envs\python38\lib\site-packages\sklearn\metrics\_classification.py:1222: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Fabrice\AppData\Local\Continuum\anaconda3\envs\python38\lib\site-packages\sklearn\metrics\_classification.py:1222: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this 

ORIG 
 {'eval_loss': 0.40837329626083374, 'eval_accuracy': 0.9075, 'eval_f1': 0.9075834882269251, 'eval_precision': 0.9086028296035125, 'eval_recall': 0.9075, 'eval_runtime': 115.9768, 'eval_samples_per_second': 65.53, 'epoch': 3.0}


...

Using custom data configuration default
Reusing dataset ag_news (C:\Users\Fabrice\.cache\huggingface\datasets\ag_news\default\0.0.0\fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a)
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly iden

Loading cached processed dataset at C:\Users\Fabrice\.cache\huggingface\datasets\ag_news\default\0.0.0\fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a\cache-4a398149fbe9330d.arrow


Step,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
10800,0.708600,0.846066,0.740012,182.018500,65.927000
21600,0.666900,0.617437,0.764425,181.590500,66.083000
32400,0.641800,0.642007,0.752128,182.184800,65.867000
43200,0.590000,0.621222,0.774508,183.075300,65.547000
54000,0.557100,0.592399,0.785612,182.812500,65.641000
64800,0.536000,0.614263,0.788200,182.839200,65.631000
75600,0.502100,0.608628,0.797116,183.427700,65.421000
86400,0.451300,0.571547,0.803165,182.530300,65.743000
97200,0.436900,0.544704,0.822890,182.139100,65.884000
108000,0.431300,0.538101,0.825614,185.144500,64.814000


ORIG 
 {'eval_loss': 32.375675201416016, 'eval_accuracy': 0.9225, 'eval_f1': 0.9224934067850175, 'eval_precision': 0.9226543311502808, 'eval_recall': 0.9225000000000001, 'eval_runtime': 115.7574, 'eval_samples_per_second': 65.655, 'epoch': 3.0}
20NG 
 {'eval_loss': 35.82275390625, 'eval_accuracy': 0.7785039941902687, 'eval_f1': 0.5826307778180877, 'eval_precision': 0.5987674268764859, 'eval_recall': 0.5690547589694692, 'eval_runtime': 106.3557, 'eval_samples_per_second': 64.736, 'epoch': 3.0}


C:\Users\Fabrice\AppData\Local\Continuum\anaconda3\envs\python38\lib\site-packages\sklearn\metrics\_classification.py:1222: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
